In [32]:
import numpy as np
syn_input_data = np.genfromtxt('Info/input.csv', delimiter=',')
syn_output_data = np.genfromtxt(
'Info/output.csv', delimiter=',').reshape([-1, 1])
letor_input_data = np.genfromtxt(
'Info/Querylevelnorm_X.csv', delimiter=',')
letor_output_data = np.genfromtxt(
'Info/Querylevelnorm_t.csv', delimiter=',').reshape([-1, 1])

In [33]:
def compute_design_matrix(X, centers, spreads):
    # use broadcast
    basis_func_outputs = np.exp(np.sum(np.matmul(X - centers, spreads) * (X - centers),axis=2)/(-2)).T
    # insert ones to the 1st col
    return np.insert(basis_func_outputs, 0, 1, axis=1)

In [39]:
def closed_form_sol(L2_lambda, design_matrix, output_data):
    return np.linalg.solve(
    L2_lambda * np.identity(design_matrix.shape[1]) +
    np.matmul(design_matrix.T, design_matrix),
    np.matmul(design_matrix.T, output_data)
    ).flatten()

In [40]:
def SGD_sol(learning_rate,minibatch_size,num_epochs,L2_lambda,design_matrix,output_data):
    N, _ = design_matrix.shape
    # You can try different mini-batch size size
    # Using minibatch_size = N is equivalent to standard gradient descent
    # Using minibatch_size = 1 is equivalent to stochastic gradient descent
    # In this case, minibatch_size = N is better
    weights = np.zeros([1, 4])
    # The more epochs the higher training accuracy. When set to 1000000,
    # weights will be very close to closed_form_weights. But this is unnecessary
    for epoch in range(num_epochs):
        for i in range(N / minibatch_size):
            lower_bound = i * minibatch_size
            upper_bound = min((i+1)*minibatch_size, N)
            Phi = design_matrix[lower_bound : upper_bound, :]
            t = output_data[lower_bound : upper_bound, :]
            E_D = np.matmul((np.matmul(Phi, weights.T)-t).T,Phi)
            E = (E_D + L2_lambda * weights) / minibatch_size
            weights = weights - learning_rate * E
            #print np.linalg.norm(E)
    return weights.flatten()

In [41]:
input_data = syn_input_data
output_data = syn_output_data
N, D = input_data.shape
# Assume we use 3 Gaussian basis functions M = 3
# shape = [M, 1, D]
centers = np.array([np.ones((D))*1, np.ones((D))*0.5, np.ones((D))*1.5])
centers = centers[:, np.newaxis, :]
# shape = [M, D, D]
spreads = np.array([np.identity(D), np.identity(D), np.identity(D)]) * 0.5
# shape = [1, N, D]
X = input_data[np.newaxis, :, :]
design_matrix = compute_design_matrix(X, centers, spreads)

In [42]:
design_matrix

array([[ 1.        ,  0.35228143,  0.78618824,  0.04522565],
       [ 1.        ,  0.27040822,  0.76142183,  0.02751353],
       [ 1.        ,  0.77751696,  0.83165481,  0.20826128],
       ..., 
       [ 1.        ,  0.5760231 ,  0.80675205,  0.11783427],
       [ 1.        ,  0.3614176 ,  0.77978949,  0.04799247],
       [ 1.        ,  0.35121316,  0.73774718,  0.04790335]])

In [47]:
# Closed-form solution
print closed_form_sol(L2_lambda=0.1,design_matrix=design_matrix,output_data=output_data)
# Gradient descent solution
print SGD_sol(learning_rate=1,minibatch_size=N,num_epochs=10000,L2_lambda=0.1,design_matrix=design_matrix,output_data=output_data)

[-3.41005736 -6.06152578  8.20880053  5.50901759]
[-3.30025941 -4.1399126   7.5356907  -0.17730426]
